# Handling Inconsistent Data

In [18]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz,process

In [3]:
data=pd.read_csv('pakistan_intellectual_capital.csv')

In [4]:
data.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [7]:
data.Country

0          Thailand
1          Thailand
2          Thailand
3          Thailand
4          Pakistan
           ...     
1137    South Korea
1138         France
1139            USA
1140       Pakistan
1141    South Korea
Name: Country, Length: 1142, dtype: object

Lets see if there are any duplicate data. When searching for duplicates, make sure you sort all elements so that way atleast it will become easier to find the duplicate values as they will appear next to each other.

In [11]:
data.Country.sort_values().unique()

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

There are a lot of problems here.
1. Germany and germany are considered to be different 
2. even if we convert all to small,' germany' and 'germany' will be an issue.
3. gaps, like south korea and southkorea

lets see if we can resolve those issues

In [14]:
data.Country=data.Country.str.lower() ## taken care of problem no 1

In [15]:
data.Country=data.Country.str.strip() ## taken care of problem no 2

In [31]:
countries_array=data.Country.sort_values().unique()

we took care of 1 and 2, but resolving pronblem 3 is quite challenging.
we have to take care of things like 
usofa and usa. For this you will need to make use of string matching. For example...

In [21]:
fuzz.token_sort_ratio('usofa','usa')

75

In [22]:
fuzz.ratio('usofa','usa')

75

In [23]:
fuzz.partial_ratio('usofa','usa')

67

In [32]:
import fuzzywuzzy as fz
fz.process.extract('south korea',countries_array,scorer=fuzz.ratio)

[('south korea', 100),
 ('southkorea', 95),
 ('austria', 44),
 ('saudi arabia', 43),
 ('scotland', 42)]

observe the above output.. 
* south korea matched 100%
* southkorea matches 95% and so on

just observe how this below code works. isin() function. isin function returns the rows 

In [39]:
data[data.Country.isin(['germany'])].head(2)

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
11,102,103,Mukhtar Hussain,University of Turbat,Computer Science,Balochistan,Lecturer,MS,SRH Hochschule Heidelberg,germany,NaN,"Semantic Web, Machine Learning",NaN
58,234,235,Qaiser Riaz,National University of Sciences and Technology,Computing,Capital,Assistant Professor,PhD,University of Bonn,germany,2015.0,"Human motion analysis, character animation, in...",NaN


In [59]:
def replace_dups(fuzzy_array,ratio=90):
    raw_data=data.Country.unique()
    countries=[country[0] for country in fuzzy_array if country[1]>ratio]
    return countries

In [84]:
fuzzy_array=fz.process.extract('south korea',countries_array,scorer=fuzz.ratio)
countries_matched=replace_dups(fuzzy_array)

In [110]:
countries_matched

['south korea', 'southkorea']

In [86]:
array

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [113]:
data.Country.isin(countries_matched)

0       False
1       False
2       False
3       False
4       False
        ...  
1137     True
1138    False
1139    False
1140    False
1141     True
Name: Country, Length: 1142, dtype: bool

In [94]:
data.loc[:,'Country']

0          thailand
1          thailand
2          thailand
3          thailand
4          pakistan
           ...     
1137    south korea
1138         france
1139            usa
1140       pakistan
1141    south korea
Name: Country, Length: 1142, dtype: object

In [114]:
rows=data.Country.isin(countries_matched)

In [116]:
data.loc[rows,['Country']] ='southkorea'

In [120]:
data.Country[data.Country=='southkorea']

33      southkorea
163     southkorea
169     southkorea
177     southkorea
189     southkorea
244     southkorea
321     southkorea
489     southkorea
677     southkorea
685     southkorea
764     southkorea
906     southkorea
931     southkorea
1137    southkorea
1141    southkorea
Name: Country, dtype: object

In [124]:
data.Country.isin(['germany'])

0       False
1       False
2       False
3       False
4       False
        ...  
1137    False
1138    False
1139    False
1140    False
1141    False
Name: Country, Length: 1142, dtype: bool

The above command says, get me the rows from country column which matches with 'germany'. return it in the form of a boolean.
Just remember that you can pass the output of isin into loc. Which is the greatest advantage

practice of the fuzzy thing

In [125]:
data.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [131]:
data.Country.loc[data.Country.isin(['germany'])]='Germany'

C:\Users\Rajat\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


see like this. Like the above code. We are passing the output of the isin into a loc. And usually that is how it is used.


In [142]:
countries_array

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [150]:
fuzz_arr=fz.process.extract('usa',countries_array)
fuzz_arr

[('usa', 100),
 ('usofa', 75),
 ('mauritius', 72),
 ('australia', 60),
 ('austria', 60)]

here again i have usofa and usa. They both mean the same thing but we ave issues

In [151]:
matching_array=[c[0] for c in fuzz_arr if c[1]>74 ]

In [152]:
matching_array

['usa', 'usofa']

In [154]:
rows=data.Country.isin(matching_array)

In [156]:
data.loc[rows,'Country']='usa'

In [161]:
data.Country.sort_values().unique()

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'southkorea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa'],
      dtype=object)

In [160]:
data.Country[data.Country=='Germany']='germany'

<ipython-input-160-b8d5e246aa57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Country[data.Country=='Germany']='germany'
